In [18]:
import pandas as pd
import requests
from glob import glob

from snowflake.connector.pandas_tools import pd_writer
import snowflake.connector.pandas_tools
import snowflake.connector
from datetime import datetime
from pathlib import Path, PurePosixPath

import os
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
ctx = snowflake.connector.connect(  user=       os.environ['USER'],
                                    password=   os.environ['PASSWORD'],
                                    account=    os.environ['ACCOUNT'],
                                    warehouse=  os.environ['WH'],
                                    database=   os.environ['DB'],
                                    schema=     os.environ['SCHEMA'],
                                    role=       os.environ['ROLE'])

# Get maximum date of data loaded so that we only import new data
a = ctx.cursor().execute("SELECT MAX(CAST(DATE/1000000 as timestamp)) as date FROM PRICE_CATCHER.raw.aeon")


In [41]:
import pandas as pd
df = a.fetch_pandas_all()
df.head()

,DATE
0,2022-01-31


In [50]:
try:
    date_existing = df.DATE.dt.date.values[0].isoformat()
except:
    date_existing = '2020-01-01' # Set default value of the above contains nothing
date_existing

'2022-01-31'

In [51]:
data_path = Path().absolute().as_posix().split('working_files')[0]  + '/working_files/data'

df_raw = pd.read_parquet(data_path + '/price_catcher_raw/aeon.parquet')
df_raw.head()

,date,premise_code,item_code,price,premise,address,premise_type,state,district,item,unit,item_group,item_category,etl_time
45138,2022-01-01,3178,1,8.89,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,AYAM BERSIH - STANDARD,1kg,BARANGAN SEGAR,AYAM,2024-05-27T02:21:30.908243
45139,2022-01-01,3178,9,35.90,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...,1kg,BARANGAN SEGAR,DAGING,2024-05-27T02:21:30.908243
45140,2022-01-01,3178,16,6.30,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,BETIK BIASA,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-27T02:21:30.908243
45141,2022-01-01,3178,18,5.90,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,PISANG BERANGAN,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-27T02:21:30.908243
45142,2022-01-01,3178,19,6.50,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,PISANG EMAS,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-27T02:21:30.908243


In [52]:
new_date = df_raw['date'].max().date().isoformat()
new_date

'2022-01-31'

In [54]:
df_to_load = df_raw[df_raw['date'] > date_existing]
print(len(df_to_load))

0


In [55]:
df_to_load

,date,premise_code,item_code,price,premise,address,premise_type,state,district,item,unit,item_group,item_category,etl_time


In [39]:
snowflake.connector.pandas_tools.write_pandas(
    conn=ctx,
    df = df_to_load,
    table_name='AEON',
    database='PRICE_CATCHER',
    schema='RAW', quote_identifiers=False
)

C:\Users\AzwanDesktop\AppData\Local\Temp\ipykernel_26240\2576786249.py:1: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  snowflake.connector.pandas_tools.write_pandas(


(True,
 1,
 3096,
 [('ujnwskjldm/file0.txt',
   'LOADED',
   3096,
   3096,
   1,
   0,
   None,
   None,
   None,
   None)])